<a href="https://colab.research.google.com/github/zip37/garrulous-octo-kumquat/blob/master/DG_conservation_law.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Librerías iniciales

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Implementaremos un solver de DG basado en el artículo de Cockburn y Shu. 

Recordemos que al discretizar usando DG se obtiene el sistema de ecuaciones

$$MU_t = L(U)$$

La matriz de masa obtenida es diagonal por bloques. Usamos polinomios de Legendre para garantizar que la matriz de masa sea diagonal, con esto ahorramos invertir la matriz y sólo nos preocuparemos por usar RK para avanzar en el tiempo, es decir, tenemos solamente:

$$U_t = L(U)$$ 

In [ ]:
#Funciones base (polinomios de Legendre)

def phi(x,degree):
  if degree == 0 :
    return 1
  elif degree == 1 :
    return x
  elif degree == 2 :
    return 0.5*(3.0*x**2-1.0)
  else:
     return 1

def phi_derivative(x,degree):
  if degree == 0 :
    return 0
  elif degree == 1 :
    return 1
  elif degree == 2 :
    return 3.0*x
  else: 
    return 0

#Flujo Lax-Friedrichs

def Nf_LF(f,Df,a,b,n): #Numerical flux, notice dependence on the "normal"
    C=max(abs(Df(a)),abs(Df(b)))
    y=0.5*((f(a)+f(b))*n-C*(b-a))
    return y

#Escribimos al operador L, contiene un sumando a integrar mediante algún tipo 
#de cuadratura, además del término para el flujo numérico.

def L(f,Df,flux,u_0,I,I_l,I_r,degree):
  quadrature = 0.0
  weights = q_weights(degree)
  points = q_points(degree)
  dx = I[1] - I[0]
  xj = 0.5*(I[1] + I[0])
  #Integracion por cuadratura gaussiana
  for i in range(degree):
    point = xj + 0.5*dx*points[i] #Pasa al intervalo [-1,1]
    increment = weights[i] * f(u_0(point)) * phi_derivative(points[i],degree)
    quadrature = quadrature + increment
  quadrature = 0.5*quadrature
  #Calculo de flujos numéricos
  numflux = flux(f,Df,I[1],I_r,1) - flux(f,Df,I_l,I[0],1)*(-1)**degree
  numflux = quadrature - numflux*(2.0*degree+1.0)/dx
  return numflux

def U_inicial(u_0,I,degree): #Integra la condición inicial
  quadrature = 0.0
  weights = q_weights(degree)
  points = q_points(degree)
  dx = I[1] - I[0]
  xj = 0.5*(I[1] + I[0])
  for i in range(degree):
    point = xj + 0.5*dx*points[i] #Pasa al intervalo [-1,1]
    increment = weights[i] * u_0(point) * phi(points[i],degree)
    quadrature = quadrature + increment
  quadrature = 0.5*(2.0*degree+1)*quadrature
  return quadrature

#Constantes para cuadratura gaussiana
def q_weights(degree):
  weights = np.zeros(degree+1)
  if degree == 0 :
    weights[0] = 2   
  elif degree == 1 :
    weights[0] = 1
    weights[1] = 1 
  elif degree == 2 :
    weights[0] = 8.0/9.0
    weights[1] = 5.0/9.0 
  else :
    weights[0] = 2 
  return weights

def q_points(degree):
  points = np.zeros(degree+1)
  if degree == 0 :
    points[0] = 0 
  elif degree == 1 :
    points[0] = 1.0/np.sqrt(3.0)
    points[1] = -points[0] 
  elif degree == 2 :
    points[0] = 0
    points[1] = np.sqrt(3.0/5.0)
    points[2] = -points[1] 
  else :
    points[0] = 0 
  return points


A continuación se tiene la implementación de un método RK para un avance en el tiempo.

In [ ]:
def RK_integrate(u_0,dt,f,Df,flux,u_0,I,I_l,I_r,degree):
  